# Import Libraries

In [1]:
import os
import pickle
import azureml.core
from azureml.core import Workspace, Dataset, Datastore
from azureml.core.environment import Environment
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice, AksWebservice, Webservice
from azureml.core.compute import ComputeTarget, AksCompute, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.exceptions import WebserviceException

from pathlib import Path

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# AzureML SDK Version

In [2]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.36.0


# WorkSpace Config

In [3]:
global ws

ws=None
try:
    ws = Workspace.from_config()
    print('Library configuration succeeded')
    print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')
except:
    print('Workspace not found')



Library configuration succeeded
aiml-workspace-dev
analytics_mart_dev_rg
northeurope
a00a8e0f-73c4-4480-8097-07bf63060185


# Environment Config

## create env from conda specification file

In [4]:
scoring_custom_env = Environment.from_conda_specification(name='scoring-conda-env', file_path='./conda_dep_scoring.yml')

## Get params

In [5]:
''' Read parameters from scoring_params config file '''
def get_params(ws):
    try:
        __path__= './params'
        PARAMS_DIR = os.path.realpath(__path__)
        print(f"PARAMS dir: {PARAMS_DIR}")

        dset1 = Dataset.get_by_name(ws, name='scoring_params')
        dset1.download(target_path=PARAMS_DIR, overwrite=True)

        import json
        params_file_path = os.path.join(PARAMS_DIR, 'parameters.json')
        with open(params_file_path) as f:
            params = json.load(f)

        response = {'status': True, 'params': params}
    except Exception as e:
        response = {'status': False, 'error' : f"Exception occurred while reading parameters from scoring config file. Error- {e}"}

    return response

params_response = get_params(ws)

params={}
if params_response['status'] == True:
    params = params_response['params']
else:
    print(params_response['error'])

' Read parameters from scoring_params config file '

PARAMS dir: /mnt/batch/tasks/shared/LS_root/mounts/clusters/rmilton4/code/Users/rmilton/scoring/params


In [6]:
AZURE_TENANT_ID = params['TENANT_ID']
AZURE_CLIENT_ID = params['PRINCIPAL_ID']
AZURE_CLIENT_SECRET = params['PRINCIPAL_PWD']
SUBSCRIPTION_ID = params['SUBSCRIPTION_ID']
RESOURCE_GP = params['RESOURCE_GP']
WORKSPACE_NAME = params['WORKSPACE_NAME']
KEY_VAULT_NAME = params['KEY_VAULT_NAME']
STORAGE_ACCOUNT = params["STORAGE_ACCOUNT"]
STORAGE_KEY = params["STORAGE_KEY"]
ANALYTICS_CONTAINER = params["ANALYTICS_CONTAINER"]
EVTDATA_FOLDER = params["EVTDATA_FOLDER"]
INFERENCE_CONTAINER = params["INFERENCE_CONTAINER"]
artwork_detail_filter_value= params["ARTWORK_DETAIL_FILTER_VALUE"]
room_detail_filter_value= params["ROOM_DETAIL_FILTER_VALUE"]
artwork_grid_filter_value= params["ARTWORK_GRID_FILTER_VALUE"]
room_grid_filter_value= params["ROOM_GRID_FILTER_VALUE"]
shuffle_grid_val = params["SHUFFLE_GRID_VAL"]
shuffle_detail_val = params["SHUFFLE_DETAIL_VAL"]


In [7]:
#Create a dictionary to set-up the env variables   
env_variables={"AZURE_TENANT_ID": AZURE_TENANT_ID,
               "AZURE_CLIENT_ID": AZURE_CLIENT_ID,
               "AZURE_CLIENT_SECRET": AZURE_CLIENT_SECRET,
               "SUBSCRIPTION_ID": SUBSCRIPTION_ID,
                "RESOURCE_GP": RESOURCE_GP,
                "WORKSPACE_NAME": WORKSPACE_NAME,
                "KEY_VAULT_NAME": KEY_VAULT_NAME,
                "STORAGE_ACCOUNT": STORAGE_ACCOUNT,
                "STORAGE_KEY": STORAGE_KEY,
                "ANALYTICS_CONTAINER": ANALYTICS_CONTAINER,
                "EVTDATA_FOLDER": EVTDATA_FOLDER,
                "INFERENCE_CONTAINER": INFERENCE_CONTAINER,
                "artwork_detail_filter_value": artwork_detail_filter_value,
                "room_detail_filter_value": room_detail_filter_value,
                "artwork_grid_filter_value": artwork_grid_filter_value,
                "room_grid_filter_value": room_grid_filter_value,
                "shuffle_grid_val": shuffle_grid_val,
                "shuffle_detail_val": shuffle_detail_val
               }
    
scoring_custom_env.environment_variables=env_variables

# Inference Config

In [8]:
global inference_config
inference_config=None

try:
    # env = Environment(name=env_name)
    # env.inferencing_stack_version = "latest"
    source_directory = os.path.realpath('.')
    
    inference_config = InferenceConfig(entry_script="score_latest.py",
                                    source_directory=source_directory,
                                    environment=scoring_custom_env                                    
                                    )

    print(f'\n\t Inference config successfully loaded.. ')
except Exception as e:
    print(f'\n\t Inference config error: {e}')


	 Inference config successfully loaded.. 


# Get respective models

In [9]:
global mdl1, mdl2, mdl3, mdl4
mdl1=None
mdl2=None
mdl3=None
mdl4=None

try:
    mdl1 = ws.models["artwork_content_model.pkl"]
    mdl2 = ws.models["artwork_collab_model.pkl"]
    mdl3 = ws.models["room_content_model.pkl"]
    mdl4 = ws.models["room_collab_model.pkl"]

    print(f'\n\t Model loaded successfully.. ') 
    print(f'\n\t {mdl1.name+"/"+str(mdl1.version)} | {mdl2.name+"/"+str(mdl2.version)} | {mdl3.name+"/"+str(mdl3.version)} | {mdl4.name+"/"+str(mdl4.version)}')
except Exception as e:
    print(f'\n\t Model load error.. {e}')



	 Model loaded successfully.. 

	 artwork_content_model.pkl/74 | artwork_collab_model.pkl/98 | room_content_model.pkl/61 | room_collab_model.pkl/39


# Deploy Model as Webservice on Azure

## Method -1: Using Aks Service

### create cluster

In [10]:
# Choose a name for inference cluster
inference_cluster_name = "test-cluster"

# Verify that the cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=inference_cluster_name)
    
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print(f'No such compute found, creating new')
    prov_config = AksCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2")
    prov_config.enable_ssl(leaf_domain_label = "contoso")

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws,
                                        name = inference_cluster_name,
                                        provisioning_configuration = prov_config)

    aks_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


### run web service

In [ ]:
aks_service_name= 'aks-mch-recommendation-v3'

try:
    deployment_config = AksWebservice.deploy_configuration(cpu_cores = 0.5,
                    memory_gb = 0.5,
                    auth_enabled = True,
                    autoscale_enabled = True,
                    enable_app_insights = True,
                    description = 'Endpoint fr Testing --- MCH Recommender Web Service using directly AKS')
    
    service = Model.deploy(ws,
                    aks_service_name,
                    models = [mdl1, mdl2, mdl3, mdl4],
                    inference_config = inference_config,
                    deployment_config = deployment_config, 
                    deployment_target = aks_target, overwrite = True)

    service.wait_for_deployment(show_output = True)
    print(f'\n\t AKS Deployment done...')
except Exception as e:
    print(f'Web Service Deployment Error: {e}')

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-25 02:51:48+00:00 Creating Container Registry if not exists.
2022-02-25 02:51:48+00:00 Registering the environment.
2022-02-25 02:51:49+00:00 Building image.

### Get existing webservice

In [12]:
aks_service = AksWebservice(ws, aks_service_name)

In [13]:
print(f'service state: {aks_service.state}')
print(aks_service.get_logs())

service state: Healthy
2022-02-18T10:51:02,116738295+00:00 - rsyslog/run 
2022-02-18T10:51:02,121710751+00:00 - iot-server/run 
2022-02-18T10:51:02,128884834+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-02-18T10:51:02,129947646+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-02-18T10:51:02,236081861+00:00 - iot-server/finish 1 0
2022-02-18T10:51:02,237727879+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (12)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 37
SPARK_HOME not set. Skipping PySpark Initialization.
NumExpr defaulting to 2 threads.
Initializing logger
2022-02-18 10:51:04,544 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2022-02-18 10:51:04,545 | root | INFO | Starting up request i

In [14]:
print(aks_service.scoring_uri)

https://contosoc9ivkd.northeurope.cloudapp.azure.com:443/api/v1/service/aks-mch-recommendation-v3/score


In [15]:
primary_key, sec_key = aks_service.get_keys()
primary_key, sec_key

('4qP9jQXbHeOgHDaxwIGXbo005R0XI6kE', '38lEm1OtX3Qka0XiDe74jhPBg39oj0Zc')

# Consume web service

In [16]:
import json, requests

# Convert the array to a serializable list in a JSON document
data = {"itemId": "139065", "userId":"045170d-2d83-ec11-8d21-000d3a64ffaa", "performedAction": "ENTER_ARTWORK"}

json_data = json.dumps(data)

# primary_key, secondary_key = aks_service.get_keys()
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ primary_key)}


endpoint = aks_service.scoring_uri

# Call the service
response = requests.post(url = endpoint,
                         data = json_data,
                         headers = headers)

# Get the predictions
predictions = response.json()
predictions

{'status': True,
 'result': [141005,
  139911,
  139195,
  139525,
  139278,
  140954,
  140204,
  139430,
  139732,
  139180,
  139405,
  138920,
  139229,
  139164,
  139395,
  139686,
  139864,
  139957,
  139931,
  140457,
  139538,
  140459,
  139185,
  139691,
  139831,
  139005,
  139681,
  140458,
  140053,
  138867,
  139084,
  139433,
  140254,
  140510,
  139412,
  140801,
  139361,
  140309,
  139203,
  139054,
  139161,
  139126,
  139904,
  140956,
  138919,
  139311,
  139799,
  139574,
  139255,
  139514,
  140608,
  139786,
  139505,
  139581,
  140201,
  139174,
  139298,
  139684,
  141154,
  139202,
  139508,
  139327,
  139241,
  139132,
  139908,
  140152,
  138970,
  139466,
  139537,
  140002,
  139239,
  139571,
  140556,
  139425,
  138860,
  138913,
  139862,
  139682,
  139061,
  139191,
  139332,
  139329,
  139579,
  139302,
  139712,
  139651,
  138963,
  140310,
  139233,
  139275,
  139769,
  140306,
  139873,
  139428,
  140008,
  139281,
  139830,
  1

In [17]:
len(predictions['result']) # ARTWORKS - 877, ROOMS - 75

876